In [1]:
import torch 
import torch.nn as nn
import numpy as np

In [2]:
n = 2
m = 1



a =  nn.Linear(n,m,bias = False)
# K = torch.Tensor([[1, 2],[2, 3]])
# a.weight = nn.Parameter(K)
print(a.weight)

Parameter containing:
tensor([[-0.1900, -0.7005]], requires_grad=True)


In [3]:
K = torch.Tensor([[1, 2],[2, 3]])
L = torch.Tensor([[1, 2],[2, 3]])

M = torch.cat((K,L),0)
print(M)

tensor([[1., 2.],
        [2., 3.],
        [1., 2.],
        [2., 3.]])


In [4]:
int(True)

1

In [19]:
S = torch.Tensor([[1, 2]])

In [24]:
torch.norm(S, p =1)

tensor(3.)

In [23]:
5**(1/2)

2.23606797749979

In [5]:
num_states = 2 # no. of states
num_inputs = 1 # no. of inputs
net_params = {}
net_params['state_dim'] = num_states
net_params['ctrl_dim'] = num_inputs
net_params['encoder_hidden_width'] = 30
net_params['encoder_hidden_depth'] = 2
net_params['encoder_output_dim'] = 20
net_params['optimizer'] = 'adam'
net_params['activation_type'] = 'tanh'
net_params['lr'] = 2e-4 # 5e-4
net_params['epochs'] = 300
net_params['batch_size'] = 256
net_params['lifted_loss_penalty'] = 0.3
net_params['l2_reg'] = 0.0         
net_params['l1_reg'] = 0.0
net_params['first_obs_const'] = True
net_params['override_C'] = True # this makes the state a part of lifted state

In [6]:
first_obs_const = int(net_params['first_obs_const']) #only one state
override_C = net_params['override_C']
if override_C:
    n_tot = int(net_params['encoder_output_dim']) + num_states + first_obs_const 
else:
    n_tot = int(net_params['encoder_output_dim']) + first_obs_const 
    
n = net_params['state_dim']
m = net_params['ctrl_dim']

In [7]:
del_A = nn.Linear(n_tot, n_tot-first_obs_const, bias = False)
del_B = nn.Linear(m*n_tot, n_tot-first_obs_const, bias = False)

In [8]:
print(del_A.weight.shape)
print(del_B.weight.shape)

torch.Size([22, 23])
torch.Size([22, 23])


In [9]:
const_obs_dyn_drift = torch.zeros((first_obs_const, n_tot))
del_A_matrix = torch.cat((const_obs_dyn_drift, del_A.weight), 0)

print(del_A_matrix.shape)

torch.Size([23, 23])


In [10]:
del_A_matrix

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1726, -0.1627, -0.1608, -0.2062,  0.0399,  0.0736, -0.0441, -0.1745,
         -0.0242, -0.0676, -0.0009, -0.0657,  0.1226, -0.0744, -0.0675,  0.0958,
          0.1382,  0.1599, -0.1601,  0.0536, -0.1383, -0.0128,  0.0024],
        [ 0.1587,  0.0291, -0.0625, -0.0930,  0.0823, -0.1110, -0.0772, -0.0411,
          0.0694,  0.1684,  0.1009,  0.1528, -0.1034,  0.0819, -0.1684, -0.0338,
         -0.0713, -0.0770, -0.1654,  0.1033, -0.1319,  0.1175, -0.0790],
        [ 0.1465,  0.0948, -0.1138, -0.0748,  0.0789,  0.0452, -0.1223,  0.1203,
         -0.1840,  0.0013, -0.1152, -0.0462, -0.0643,  0.1476, -0.0639, -0.1256,
         -0.2073,  0.1216, -0.0277,  0.0305,  0.1094, -0.1646, -0.1325],
        [-0.1567,  0.1999,  0.1308, -0.0371,  0.2005, -0.020

In [11]:
del_A_matrix_np = np.array(del_A_matrix.detach())
print(del_A_matrix_np.shape)

(23, 23)


In [12]:
del_A_matrix_np

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.1725971 , -0.16267058, -0.16075514, -0.2062203 ,  0.0399129 ,
         0.07358263, -0.044063  , -0.17448458, -0.02416556, -0.06762171,
        -0.00086328, -0.06572739,  0.12259204, -0.07443333, -0.06751783,
         0.0957787 ,  0.1381879 ,  0.15985075, -0.16010998,  0.05361233,
        -0.13828254, -0.01280625,  0.00242737],
       [ 0.15867853,  0.02914061, -0.062525  , -0.09302785,  0.082257  ,
        -0.11096023, -0.07717367, -0.04109318,  0.06936924,  0.16837735,
         0.10089069,  0.15276583, -0.10341142,  0.08186789, -0.16839221,
        -0.03382041, -0.07126746, -0.07701116, -0.16540459,  0.10333949,
        -0.13186519,  0.1174

In [13]:
del_A_prev = torch.from_numpy(del_A_matrix_np[int(first_obs_const):,:])
print(del_A_prev.shape)

torch.Size([22, 23])


In [14]:
del_A.weight = nn.Parameter(del_A_prev)
print(del_A.weight.shape)

torch.Size([22, 23])


In [15]:
print(del_A.weight)

Parameter containing:
tensor([[ 0.1726, -0.1627, -0.1608, -0.2062,  0.0399,  0.0736, -0.0441, -0.1745,
         -0.0242, -0.0676, -0.0009, -0.0657,  0.1226, -0.0744, -0.0675,  0.0958,
          0.1382,  0.1599, -0.1601,  0.0536, -0.1383, -0.0128,  0.0024],
        [ 0.1587,  0.0291, -0.0625, -0.0930,  0.0823, -0.1110, -0.0772, -0.0411,
          0.0694,  0.1684,  0.1009,  0.1528, -0.1034,  0.0819, -0.1684, -0.0338,
         -0.0713, -0.0770, -0.1654,  0.1033, -0.1319,  0.1175, -0.0790],
        [ 0.1465,  0.0948, -0.1138, -0.0748,  0.0789,  0.0452, -0.1223,  0.1203,
         -0.1840,  0.0013, -0.1152, -0.0462, -0.0643,  0.1476, -0.0639, -0.1256,
         -0.2073,  0.1216, -0.0277,  0.0305,  0.1094, -0.1646, -0.1325],
        [-0.1567,  0.1999,  0.1308, -0.0371,  0.2005, -0.0209,  0.0160,  0.0571,
         -0.1303, -0.1029, -0.0472, -0.1977, -0.0763, -0.0517,  0.0784, -0.1296,
         -0.0320,  0.0260,  0.1962,  0.0052, -0.1604,  0.1371, -0.1129],
        [ 0.0810,  0.1648,  0.0075, -0